# Подготовка данных для обучения моделей

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* https://pytorch.org/docs/stable/data.html
* https://pytorch.org/tutorials/beginner/data_loading_tutorial.html
* Deep Learning with PyTorch (2020) Авторы: Eli Stevens, Luca Antiga, Thomas Viehmann


## Задачи для совместного разбора

1. Создайте синтетический датасет для задачи регрессии и представьте его в виде `torch.utils.data.Dataset`

In [340]:
import torch as th
from torch.utils.data import Dataset
from sklearn.datasets import make_regression

In [341]:
X, y = make_regression(n_samples=1000, n_features=10)
X.shape, y.shape, type(X)

((1000, 10), (1000,), numpy.ndarray)

In [342]:
from typing import Callable

class RegressionDataset(Dataset):
  def __init__(self, transform: Callable | None = None, **kwargs):
    super().__init__()
    self.X, self.y = make_regression(**kwargs)
    self.transform = transform

  def custom_method(self):
    ...

  def __getitem__(self, idx):
    x = self.X[idx]
    if self.transform is not None:
      x = self.transform(x)
    y = self.y[idx]

    return x, y

  def __len__(self):
    return len(self.X)

In [343]:
def f(x):
  return x

f(5)

5

In [344]:
class MyCallable:
  def __call__(self, x):
    return x

In [345]:
c = MyCallable()
c(5)

5

In [346]:
dataset = RegressionDataset(n_samples=1000, n_features=10)
dataset[0]

(array([-1.32797131,  0.82477573, -0.01724446, -1.38313809, -0.67764828,
        -0.4715914 , -1.52672299, -0.4872484 ,  0.02235929,  1.50440413]),
 -171.63392100324702)

In [347]:
import numpy as np


def my_transformer(x: np.ndarray) -> np.ndarray:
  return 1000 * x

In [348]:
class MyCallable:
  def __init__(self, coef: int) -> None:
    self._coef = coef

  def __call__(self, x: np.ndarray) -> np.ndarray:
    return self._coef * x

In [349]:
dataset = RegressionDataset(
    transform=my_transformer,
    n_samples=1000,
    n_features=10
)
dataset[0]

(array([1066.90136867,  942.1223179 , 1820.74100405, -560.69033818,
        1503.03900502, -631.73687489, -655.99575706, -318.86003369,
         742.57409338, -949.79548129]),
 170.30761755408827)

In [350]:
dataset = RegressionDataset(
    transform=MyCallable(coef=10000),
    n_samples=1000,
    n_features=10
)
dataset[0]

(array([ -2294.00492106,   6818.43741826,  -1231.01947547,  -8360.3242784 ,
          2558.83872393,    391.35546981,  -7364.26212882,   6122.74929242,
         -3975.06620669, -10416.58692466]),
 -50.95546519322443)

In [351]:
dataset[:2]

(array([[ -2294.00492106,   6818.43741826,  -1231.01947547,
          -8360.3242784 ,   2558.83872393,    391.35546981,
          -7364.26212882,   6122.74929242,  -3975.06620669,
         -10416.58692466],
        [ -4964.12578053,   4058.98351689,   5551.75745219,
          23089.92488003,  12388.6240451 ,  -1018.43395151,
         -14678.15835362,  -6764.4334356 ,  10235.59643866,
          -8093.56082336]]),
 array([-50.95546519, 150.67558805]))

In [352]:
from torch.utils.data import random_split

In [353]:
train, val, test = random_split(dataset, lengths=[0.7, 0.15, 0.15])

In [354]:
train[0]

(array([-22336.20308869,   1971.96483699, -11471.65831817,   8873.55448203,
         -1815.08060433,   9307.70619217,  -7151.35057232,  26070.96865866,
          -987.66717921,  -1118.86195618]),
 206.88014971154604)

In [355]:
len(train)

700

In [356]:
from torch.utils.data import DataLoader

In [357]:
loader = DataLoader(train, 64, )
iter(loader)

In [358]:
loader = DataLoader(train, 64, )

it = iter(loader)
x, y = next(it)
x.shape, y.shape

(torch.Size([64, 10]), torch.Size([64]))

In [359]:
for x, y in loader:
  print(x.shape, y.shape)
  # break

torch.Size([64, 10]) torch.Size([64])
torch.Size([64, 10]) torch.Size([64])
torch.Size([64, 10]) torch.Size([64])
torch.Size([64, 10]) torch.Size([64])
torch.Size([64, 10]) torch.Size([64])
torch.Size([64, 10]) torch.Size([64])
torch.Size([64, 10]) torch.Size([64])
torch.Size([64, 10]) torch.Size([64])
torch.Size([64, 10]) torch.Size([64])
torch.Size([64, 10]) torch.Size([64])
torch.Size([60, 10]) torch.Size([60])


In [360]:
700 % 64

60

## Задачи для самостоятельного решения

In [361]:
import pandas as pd
import torch as th
from torch.utils.data import DataLoader, Dataset, random_split
from typing import Callable
from sklearn.preprocessing import OrdinalEncoder
from typing import Any

<p class="task" id="1"></p>

1\. Считайте файл `bank-full.csv` ([источник](https://www.kaggle.com/datasets/hariharanpavan/bank-marketing-dataset-analysis-classification)) в виде `pd.DataFrame`.

Опишите класс `BankDatasetBase`. Решение должно удовлетворять следующим критериям:

* класс наследуется от `torch.utils.data.Dataset`;
* при создании объекта в конструктор передается набор данных в виде `pd.DataFrame`;
* объекты класса имеют поля `X` и `y` с признаками и метками соответственно;
* класс реализует интерфейс последовательностей (`__getitem__` + `__len__`);
* `obj[i]` возвращает кортеж, содержащий `i`-ую строку из `obj.X` (серию) и `i`-ую строку из `obj.y` (строку).
    
Создайте объект класса `BankDatasetBase` и продемонстрируйте работоспособность.

- [ ] Проверено на семинаре

In [362]:
data = pd.read_csv('bank-full.csv')
data.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [363]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  int64 
 1   job        45211 non-null  object
 2   marital    45211 non-null  object
 3   education  45211 non-null  object
 4   default    45211 non-null  object
 5   balance    45211 non-null  int64 
 6   housing    45211 non-null  object
 7   loan       45211 non-null  object
 8   contact    45211 non-null  object
 9   day        45211 non-null  int64 
 10  month      45211 non-null  object
 11  duration   45211 non-null  int64 
 12  campaign   45211 non-null  int64 
 13  pdays      45211 non-null  int64 
 14  previous   45211 non-null  int64 
 15  poutcome   45211 non-null  object
 16  y          45211 non-null  object
dtypes: int64(7), object(10)
memory usage: 5.9+ MB


In [364]:
class BankDatasetBase(Dataset):
    def __init__(self, data: pd.DataFrame) -> None:
        super().__init__()
        self.X, self.y = data.drop('y', axis=1), data.y
        
    def __getitem__(self, idx: int) -> tuple:
        return self.X.iloc[idx], self.y.iloc[idx]

    def __len__(self) -> int:
        return self.y.shape[0]

In [365]:
bdb = BankDatasetBase(data)
len(bdb)

45211

In [366]:
bdb[0]

(age                  58
 job          management
 marital         married
 education      tertiary
 default              no
 balance            2143
 housing             yes
 loan                 no
 contact         unknown
 day                   5
 month               may
 duration            261
 campaign              1
 pdays                -1
 previous              0
 poutcome        unknown
 Name: 0, dtype: object,
 'no')

<p class="task" id="2"></p>

2\. Опишите класс `BankDataset`. Решение должно удовлетворять всем критериям из предыдущего задания, а также:
* при создании объекта в конструктор может быть передан необязательные аргументы `transform` и `target_transform`;
* если аргумент `transform` был передан, то при получении `i`-го элемента, нужно вызвать `transform(x)` и вернуть полученный результат.
* если аргумент `target_transform` был передан, то при получении `i`-го элемента, нужно вызвать `target_transform(y)` и вернуть полученный результат.

Создайте объект класса `BankDataset` и продемонстрируйте работоспособность (без передачи `target_transform` и `transform`).

- [ ] Проверено на семинаре

In [367]:
class BankDataset(Dataset):
    def __init__(
            self,
            data: pd.DataFrame,
            transform: Callable | None = None,
            target_transform: Callable | None = None
    ) -> None:
        super().__init__()
        self.X, self.y = data.drop('y', axis=1), data.y
        self.transform = transform
        self.target_transform = target_transform

    def __getitem__(self, idx: int) -> tuple:
        x = self.X.iloc[idx]
        y = self.y.iloc[idx]
        if self.transform is not None:
            x, y  = self.transform(x, y)
        if self.target_transform is not None:
            x, y = self.target_transform(x, y)
        return x, y

    def __len__(self) -> int:
        return self.y.shape[0]

In [368]:
bd = BankDataset(data)
len(bd)

45211

In [369]:
bd[0]

(age                  58
 job          management
 marital         married
 education      tertiary
 default              no
 balance            2143
 housing             yes
 loan                 no
 contact         unknown
 day                   5
 month               may
 duration            261
 campaign              1
 pdays                -1
 previous              0
 poutcome        unknown
 Name: 0, dtype: object,
 'no')

<p class="task" id="3"></p>

3\. Опишите класс `OrdinalEncoderTransform`. Решение должно удовлетворять следующим критериям:

* при создании объекта в конструктор передаются названия нечисловых столбцов в датасете
* класс реализует интерфейс `Callable` (`__call__`); метод `__call__` имеет один параметр (признаки) и возвращает набор признаков, в котором нечисловые характеристики закодированы целыми числами;
* состояние объекта (индексы для кодирования) обновляется в момент очередного вызова `__call__` (т.е. все данные сразу никогда не передаются никакому методу объекта).
    
Продемонстрируйте работоспособность, создав объект `BankDataset` и передав при создании объект класса `OrdinalEncoderTransform`.

- [ ] Проверено на семинаре

In [370]:
class OrdinalEncoderTransform:
    def __init__(self, category_columns: list[str]) -> None:
        self.category_columns = category_columns
        self.category_indexers = {col: {} for col in category_columns}

    def __call__(self, x: pd.Series, y: str) -> pd.Series:
        for col in self.category_columns:
            if col in x:
                if x[col] not in self.category_indexers[col]:
                    self.category_indexers[col][x[col]] = len(self.category_indexers[col])
                x[col] = self.category_indexers[col][x[col]]
        return x, y

In [371]:
pd.options.mode.copy_on_write = True
cats = data.drop('y', axis=1).select_dtypes(include=['object', 'category']).columns
encoder = OrdinalEncoderTransform(cats)
bd = BankDataset(data, transform=encoder)
bd[0]

(age            58
 job             0
 marital         0
 education       0
 default         0
 balance      2143
 housing         0
 loan            0
 contact         0
 day             5
 month           0
 duration      261
 campaign        1
 pdays          -1
 previous        0
 poutcome        0
 Name: 0, dtype: object,
 'no')

<p class="task" id="4"></p>

4\. Опишите класс `LabelEncoderTransform`. Решение должно удовлетворять следующим критериям:

* класс реализует интерфейс `Callable` (`__call__`); метод `__call__` имеет один параметр (строку) и возвращает целое число, соответствующее этой строке;
* состояние объекта (индексы для кодирования) обновляется в момент очередного вызова `__call__` (т.е. все данные сразу никогда не передаются никакому методу объекта).
    
Продемонстрируйте работоспособность, создав объект `BankDataset` и передав при создании объекта в качестве аргумента `target_transform` объект класса `LabelEncoderTransform`.

- [ ] Проверено на семинаре

In [372]:
class LabelEncoderTransform:
    def __init__(self) -> None:
        self.label_to_index = {}
        
    def __call__(self, X: pd.Series, label: str) -> int:
        if label not in self.label_to_index:
            self.label_to_index[label] = len(self.label_to_index)
        return X, self.label_to_index[label]

In [373]:
label_encoder = LabelEncoderTransform()
bd = BankDataset(data, target_transform=label_encoder)
bd[0]

(age                  58
 job          management
 marital         married
 education      tertiary
 default              no
 balance            2143
 housing             yes
 loan                 no
 contact         unknown
 day                   5
 month               may
 duration            261
 campaign              1
 pdays                -1
 previous              0
 poutcome        unknown
 Name: 0, dtype: object,
 0)

<p class="task" id="5"></p>

5\. Опишите класс `ToTensor`.  Решение должно удовлетворять следующим критериям:
* класс реализует интерфейс `Callable` (`__call__`); метод `__call__` принимает на вход серию или фрейм и возвращает тензор.

Опишите класс `Compose`.  Решение должно удовлетворять следующим критериям:
* при создании объекта в конструктор передается список объектов `transforms`, каждый из которых имеет метод `__call__(x, y)`;
* класс реализует интерфейс `Callable` (`__call__`); метод `__call__` принимает имеет параметра (признаки и класс в числовом виде) и и возвращает кортеж, полученный путем последовательного вызова объектов из `transforms`.

Продемонстрируйте работоспособность, создав объект `BankDataset` и передав при создании преобразования `Compose` список из объектов LabelEncoderTransform и ToTensor.

- [ ] Проверено на семинаре

In [374]:
class ToTensor:
    def __call__(self, X: pd.Series | int, y: str) -> th.Tensor:
        return th.tensor(X.values.astype(float), dtype=th.float32), y

class Compose:
    def __init__(self, transforms: list[Transform]) -> None:
        self.transforms = transforms

    def __call__(self, X: Any, y: Any) -> Any:
        for transform in self.transforms:
            if isinstance(transform, LabelEncoderTransform):
                X, y = transform(X, y)
            else:
                X, y = transform(X, y)
        return X, y

In [375]:
comp = Compose([OrdinalEncoderTransform(cats), LabelEncoderTransform(), ToTensor()])
bd = BankDataset(data, transform=comp)
bd[0]

(tensor([ 5.8000e+01,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          2.1430e+03,  0.0000e+00,  0.0000e+00,  0.0000e+00,  5.0000e+00,
          0.0000e+00,  2.6100e+02,  1.0000e+00, -1.0000e+00,  0.0000e+00,
          0.0000e+00]),
 0)

<p class="task" id="6"></p>

6\. Разделите датасет из предыдущего задания на обучающую и тестовую выборку в соотношении 75% на 25%. Создайте объект `DataLoader` для получения пакетов размера 64, полученных из перемешанного обучающего датасета. Кастомизируйте `DataLoader` таким образом, чтобы пакет признаков был представлен в виде трехмерного тензора размера 64x2x8 (разделите 16 признаков на два тензора по 8). Получите один пакет и выведите на экран размерность тензоров пакета.

- [ ] Проверено на семинаре

In [376]:
def collate_fn(batch):
    x, y = zip(*batch)
    x = th.stack(x).view(-1, 2, 8)
    y = th.tensor(y)
    return x, y

In [377]:
train_data, test_data = random_split(bd, lengths=[0.75, 0.25])
train_loader = DataLoader(train_data, shuffle=True, batch_size=64, collate_fn=collate_fn)
it = iter(train_loader)
x, y = next(it)
for X_batch, y_batch in train_loader:
    print(X_batch.shape)  
    print(y_batch.shape)  

torch.Size([64, 2, 8])
torch.Size([64])
torch.Size([64, 2, 8])
torch.Size([64])
torch.Size([64, 2, 8])
torch.Size([64])


torch.Size([64, 2, 8])
torch.Size([64])
torch.Size([64, 2, 8])
torch.Size([64])
torch.Size([64, 2, 8])
torch.Size([64])
torch.Size([64, 2, 8])
torch.Size([64])
torch.Size([64, 2, 8])
torch.Size([64])
torch.Size([64, 2, 8])
torch.Size([64])
torch.Size([64, 2, 8])
torch.Size([64])
torch.Size([64, 2, 8])
torch.Size([64])
torch.Size([64, 2, 8])
torch.Size([64])
torch.Size([64, 2, 8])
torch.Size([64])
torch.Size([64, 2, 8])
torch.Size([64])
torch.Size([64, 2, 8])
torch.Size([64])
torch.Size([64, 2, 8])
torch.Size([64])
torch.Size([64, 2, 8])
torch.Size([64])
torch.Size([64, 2, 8])
torch.Size([64])
torch.Size([64, 2, 8])
torch.Size([64])
torch.Size([64, 2, 8])
torch.Size([64])
torch.Size([64, 2, 8])
torch.Size([64])
torch.Size([64, 2, 8])
torch.Size([64])
torch.Size([64, 2, 8])
torch.Size([64])
torch.Size([64, 2, 8])
torch.Size([64])
torch.Size([64, 2, 8])
torch.Size([64])
torch.Size([64, 2, 8])
torch.Size([64])
torch.Size([64, 2, 8])
torch.Size([64])
torch.Size([64, 2, 8])
torch.Size([64])
